# Exercise Set 13: Network formation


In this Exercise Set 13 we will investigate network formation among high school pupils. 

## Part 1: Network formation


Load the data using the script below. Read a bit about the dataset [here](http://www.sociopatterns.org/datasets/high-school-contact-and-friendship-networks/) to get an understanding of what is in each variable. 

The script gives you two dataframes to work with: 
 > `el`, which is an edge-list 
 >
 > `ind` which contains individual characteristics

In [1]:
import networkx as nx
import numpy as np
import pandas as pd

url_base = 'http://www.sociopatterns.org/wp-content/uploads/2015/'

# edgelist
url_el = url_base + '07/High-School_data_2013.csv.gz'
col_names_el = ['timestamp', 'u1', 'u2', 'class1', 'class2']
el = pd.read_csv(url_el, header=None, names=col_names_el, delimiter=' ')

# individual characteristics
url_ind = url_base + '09/metadata_2013.txt'
col_names_ind = ['u', 'class', 'gender']
ind = pd.read_csv(url_ind, header=None, names=col_names_ind, delimiter='\t')\
            .set_index('u')

# remove observation with missing gender
has_gender = ind[ind.gender!='Unknown'].index

# DataFrames
ind = ind.loc[has_gender].copy()
el = el[el.u1.isin(has_gender) &  el.u2.isin(has_gender)].copy()

> **Ex. 13.1.1**: Describe the edgelist columns content. Parse the timestamp. What is the resolution of meetings? Use the parsed timestamp to count the meetings by hour in local time.

**Answer:** The `el` dataframe contains rows representing meetings (links in the network) between two people (nodes in the network). The dataframe has 5 columns:
- `timestamp`: A timestamp for the time of the link.
- `u1`: The identifier for the first person (node).
- `u2`: The identifier for the second person (node).
- `class1`: The class of the first person.
- `class2`: The class of the second person.

The resolution is 20 second, meaning that the two people were in contact within a 20-second interval.

In [2]:
# Convert unix timestamp to datetime
el['timestamp'] = pd.to_datetime(el['timestamp'], unit='s', origin='unix')

# Create new column containing hour
el['hour'] = el['timestamp'].dt.hour

# Count meetings by hour
el.groupby('hour')['timestamp'].count()

hour
7     19628
8     21227
9     27457
10    18390
11    21398
12    19293
13    20363
14    17541
15    16352
Name: timestamp, dtype: int64

In [3]:
# Alternatively: Count meetings by day and hour
el['day-hour'] = el['timestamp'].dt.strftime('%d-%H')
el.groupby('day-hour')['timestamp'].count()

day-hour
02-11    5556
02-12    4259
02-13    6617
02-14    5715
02-15    5972
03-07    6048
03-08    5286
03-09    7104
03-10    5096
03-11    4675
03-12    4193
03-13    5172
03-14    3772
03-15    4316
04-07    5100
04-08    6218
04-09    7309
04-10    4013
04-11    3998
04-12    4555
04-13    3109
04-14    2567
04-15    2117
05-07    4603
05-08    4851
05-09    6146
05-10    4230
05-11    3063
05-12    3039
05-13    3680
05-14    3461
05-15    2595
06-07    3877
06-08    4872
06-09    6898
06-10    5051
06-11    4106
06-12    3247
06-13    1785
06-14    2026
06-15    1352
Name: timestamp, dtype: int64

> **Ex. 13.1.2**: Count the number of meetings for each edge and save this as a DataFrame called `el_agg`. Filter out edges with less than 5 minutes of meetings. Attach the gender and class of both nodes.

In [4]:
# Create column with person 1 and 2
el['u1-u2'] = el['u1'].astype(str) + '-' + el['u2'].astype(str)

# Count the number of meetings for each edge
us_count = el.groupby('u1-u2').count()['timestamp']

# Make dataframe with each edge and the number of meetings for each edge
u1s = [i.split('-')[0] for i in us_count.index]
u2s = [i.split('-')[1] for i in us_count.index]
el_agg = pd.DataFrame(list(zip(u1s, u2s, us_count)), columns = ['u1','u2','meet_count'])

# Filter out edges with less than 5 minutes of meeting (meet count < 15, since resolution is 20 seconds (15/3))
el_agg = el_agg[el_agg['meet_count'] >= 15]

# Get class and gender for each person
def get_class(u):
    return ind.loc[int(u)]['class']

def get_gender(u):
    return ind.loc[int(u)]['gender']

el_agg['u1_class'] = el_agg['u1'].apply(get_class)
el_agg['u2_class'] = el_agg['u2'].apply(get_class)
el_agg['u1_gender'] = el_agg['u1'].apply(get_gender)
el_agg['u2_gender'] = el_agg['u2'].apply(get_gender)

> **Ex. 13.1.3**: Answer question in the function `fraction_triangles` below. Explain how `fraction_triangles` is related to  computing the clustering coefficient (using `nx.average_clustering`).
>
>> *Hint:* The following code does the same thing as `fraction_triangles`, but at a scale where you can understand what's going on. If you have a hard time understanding the code in the function you can try to play around with this simpler example
>>
>> ```python
>> import networkx as nx 
>>
>> A  = np.array(
>>     [[0, 1, 1, 0],
>>      [1, 0, 1, 0],
>>      [1, 1, 0, 1],
>>      [0, 0, 1, 0]]
>> )
>>
>> G = nx.from_numpy_array(A)
>> nx.draw(G,with_labels=True)
>>
>> def nth(A, n):
>>     A_ = A.copy()    
>>     for _ in range(1,n):
>>         A = A.dot(A_)
>>     return A
>>
>> a_t = nth(A,3).diagonal().sum()/6
>> n = len(A[:,0])
>> p_t = binom(n, 3)
>> ```


In [5]:
def make_net(el_, nodes):
    '''
    Convert edgelist to networkx graph which is 
    binary and undirected.
    
    Parameters
    ----------
    el_ : DataFrame
        Table containing an edgelist with columns 
        `u1` and `u2` which are the nodes in the edge.
        
    nodes : array-like
        1d array containing the node identities.
    '''    
    
    nx_input = el_, 'u1', 'u2', 'meet_count', nx.Graph()
    g = nx.from_pandas_edgelist(*nx_input)
    g.add_nodes_from(nodes)
    return g

In [6]:
from scipy.special import binom

def fraction_triangles(el_, nodes):
    '''
    Compute fraction of actual triangles out 
    of the potential triangles.
    
    Parameters
    ----------
    el_ : DataFrame
        Table containing an edgelist with columns 
        `u1` and `u2` which are the nodes in the edge.
        
    nodes : array-like
        1d array containing the node identities.
    '''
    
    g = make_net(el_, nodes)
    
    #Q.1: what is `A`?: The adjacency matrix which is symmetric and binary
    #Q.2: what does `A**3` do? Compute the number of paths between two nodes
    #Q.3: what is diagonal of A_t? The number of actual paths of length 3, 
    # i.e. triangles, which include the person. these are called cycles
    # because they start and end at the same person
    
    # count actual triangles    
    A = nx.to_scipy_sparse_matrix(g)
    A_t = A**3
    a_t = A_t.diagonal().sum()/6
    
    #Q.4: what does `binom(n,3)` compute? The number of triangles including the person
    
    # count potential triangles
    n = len(g.nodes())
    p_t = binom(n, 3)
        
    return a_t/p_t

**ANSWER:** The clustering coefficient of a node the clustering of a node is the fraction of possible triangles through that node that exist (which can be computed by `fraction_triangles`). The average clustering coefficient of the network (`nx.average_clustering`) is the average of this over all the nodes in the network.

> **Ex. 13.1.4**: Apply the function `fraction_triangles` to `el_agg` and print the triangle fraction in the network. Next remove all edges that go between classes. Compute triangle fraction within each class and store it. Compute the mean within class triangles and bootstrap the standard error of the mean. Comment on the output.
>
>> *Hint:* To bootstrap an estimate draw $k>>0$ samples with replacement from the data. Compute the estimate on each of these samples and average them in the end to get the bootstrapped estimate. 

In [7]:
# Print triangle fraction
us = list(set(el_agg['u1']).union(set(el_agg['u2'])))
trifrac = fraction_triangles(el_agg, us)
print('Triangle fraction: {}'.format(round(trifrac,4)))

# Get only the meeting where both students are in the same class
el_agg_c = el_agg[el_agg['u1_class']==el_agg['u2_class']]
classes = list(set(el_agg['u1_class']).union(set(el_agg['u2_class'])))

Triangle fraction: 0.0003


In [8]:
# Calculate triangle fraction for each class and store in dictionary
def get_class_triangles(el_agg_c):
    class_triangles = dict()
    for _class in classes:
        df = el_agg_c[el_agg_c['u1_class'] == _class]
        us_df = list(set(df['u1']).union(set(df['u2'])))
        trifrac = fraction_triangles(df, us_df)
        class_triangles[_class] = trifrac
    return class_triangles
class_triangles = get_class_triangles(el_agg_c)

In [9]:
# Bootstrap class triangles k times
k = 10
bs_dict = dict()
for _class in classes:
    bs_dict[_class] = []
for i in range(k):
    bs_triangles = get_class_triangles(el_agg_c.sample(n=len(el_agg_c),replace=True))
    for _class in classes:
        bs_dict[_class].append(bs_triangles[_class])
        
# Store standard errors
class_ses = dict()
for _class in classes:
    class_ses[_class] = np.std(bs_dict[_class])
    
# Print triangle fractions (in percent) and bootstraped standard errors
for _class in classes:
    print('Triangle fraction for {}: {} ± {}'.format(_class, round(class_triangles[_class],3),
                                                    round(class_ses[_class],3)))

# COMMENT: There appears to be a significant difference in the triangle fractions across classes. For instance,
#          2BIO3 has significantly more triangles than PSI*.

Triangle fraction for 2BIO3: 0.035 ± 0.002
Triangle fraction for PSI*: 0.011 ± 0.001
Triangle fraction for 2BIO2: 0.034 ± 0.001
Triangle fraction for MP*2: 0.028 ± 0.002
Triangle fraction for PC*: 0.02 ± 0.002
Triangle fraction for MP*1: 0.016 ± 0.002
Triangle fraction for 2BIO1: 0.013 ± 0.001
Triangle fraction for PC: 0.023 ± 0.001
Triangle fraction for MP: 0.03 ± 0.002


Recall from class that we can define the following measures of homophily. We define **homophily index** inspired by [Currarini et al. (2009)](https://doi.org/10.2139/ssrn.1021650):
- share of edges that are same type: $H = \frac{s}{s+d}$
- possible range [0,1]


We define **baseline homophily** as: 
- We count fraction of potential edges in population of nodes which are same type:

\begin{equation}B=\frac{\sum_t\#potential(n_t)}{\#potential(n)}, \qquad \#potential(k)=\frac{k\cdot(k-1)}{2}\end{equation}

- Interpretation: Expected homophily from random link formation.     

We define **inbreeding homophily** as:      

\begin{equation}IH=\frac{H-B}{1-B}\end{equation}


> **Ex. 13.1.5**: Compute the inbreeding homophily for each class. Use the class measures to compute the mean. Use a bootstrap to compute whether there is inbreeding homophily.

In [10]:
# Calculate inbreeding homophily index for each class
def calc_inho(el_agg_c): 
    class_hopi = []
    for _class in classes:
        # Calculate homophily index
        df = el_agg_c[el_agg_c['u1_class'] == _class]
        n_total = df['meet_count'].sum()
        n_same_gender = df[df['u1_gender']==df['u2_gender']]['meet_count'].sum()
        hopi = n_same_gender/n_total
        
        # Calculate males and females in network for use in computing baseline homophily
        us_c = list(set(df['u1']).union(set(df['u2'])))
        ms, fs = 0, 0
        for u in us_c:
            if ind.loc[int(u)]['gender'] == 'F':
                fs = fs + 1
            else:
                ms = ms + 1
        
        # Calculate baseline homophily
        upper = ms*(ms-1)/2 + fs*(fs-1)/2
        lower = (ms+fs)*(ms+fs-1)/2
        ba_hopi = upper/lower

        # Store inbreeding homophily for classes
        class_hopi.append((hopi-ba_hopi)/(1-ba_hopi))
    return np.mean(class_hopi)
    
# Print mean class inbreeding homophily
inho = calc_inho(el_agg_c)
print(round(inho,3))

0.243


In [11]:
# Bootstrap calculation of fraction of triangles k times
k = 100
inhos = [calc_inho(el_agg_c.sample(n=len(el_agg_c),replace=True)) for i in range(k)]
se = np.std(inhos)
print('The average inbreeding homophily is: {} ± {}'.format(round(inho,3),round(se,3)))

# CONCLUSION: There is inbreeding homophily, meaning that people of the same gender tend to form links more often
#             than if link formation were random.

The average inbreeding homophily is: 0.243 ± 0.06


> **Ex. 13.1.6** (BONUS): Describe what an unsupported edge is. Construct a test of whether there is a preference for forming  triangles within same gender than across.
>
>> *Hint:*  You can find inspiration in the approach of [Chandrasekhar, Jackson (2018)](https://web.stanford.edu/~arungc/CJ_sugm.pdf) pp. 31-35. They construct an almost identical test for triangle formation across castes in Indian villages.

In [12]:
# [Answer to ex. 13.1.6 here]